In [27]:
import pandas as pd
train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')
print(train_df.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [28]:
import numpy as np
import cv2

def get_img_feat(img):
    feats = [np.mean(img),np.std(img),np.median(img),np.max(img),np.min(img)]
    return feats
    
def get_other_feat(df):
    feats = []
    for i, row in df.iterrows():
        img1 = np.array(row['band_1']).reshape(75, 75).astype('float32')
        img2 = np.array(row['band_2']).reshape(75, 75).astype('float32')
        if row['inc_angle'] == 'na':
            ang = 0
        else:
            ang = float(row['inc_angle'])
        feats.append(get_img_feat(img1)+get_img_feat(img2)+[ang])
    return np.array(feats).astype('float32')

train_feat = get_other_feat(train_df)
test_feat = get_other_feat(test_df)

In [29]:
print(train_feat[:5])  

[[ -2.79110451e+01   2.76453733e+00  -2.78799210e+01  -2.13148996e-01
   -3.82113762e+01  -2.99101162e+01   2.38128400e+00  -3.00078468e+01
   -1.12521534e+01  -4.11359177e+01   4.39239006e+01]
 [ -1.35665531e+01   3.14253163e+00  -1.36541986e+01   1.25704088e+01
   -2.31253090e+01  -2.53591061e+01   2.93409801e+00  -2.54861298e+01
    4.40520011e-02  -3.47658310e+01   3.81562004e+01]
 [ -2.30536995e+01   2.22390556e+00  -2.29350185e+01  -9.91847706e+00
   -3.33911972e+01  -2.48398228e+01   2.12527490e+00  -2.46062050e+01
   -1.56058788e+01  -3.41488190e+01   4.52859001e+01]
 [ -2.32107716e+01   2.56623268e+00  -2.33032379e+01   4.79562712e+00
   -3.22041359e+01  -2.95679111e+01   2.38997960e+00  -2.95499229e+01
   -5.55451584e+00  -3.95640526e+01   4.38306007e+01]
 [ -2.52800274e+01   2.30528760e+00  -2.52066154e+01  -6.95603609e+00
   -3.50104866e+01  -2.56277275e+01   2.27624393e+00  -2.54669209e+01
   -9.43400574e+00  -4.02761154e+01   3.56255989e+01]]


In [30]:
train_feat.shape

(1604, 11)

In [31]:
train_feat.dtype

dtype('float32')

In [32]:
import pickle
with open('../features/other_feat.pkl','wb') as fout:
    pickle.dump([train_feat,test_feat],fout)